### 2-2-LiveBeeContourFinder.ipynb

In [5]:
from pathlib import Path
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import statistics
import os
import cv2
import time

from PIL import Image
from scipy.ndimage import binary_fill_holes

In [6]:
# Define directories
data_dir = Path("/mnt/g/Projects/Master/Data/")

input_dir = data_dir / "Processed" / "LiveBees" / "2-LiveWingLabelCropsManuallyImproved" / "Labels"
output_dir = data_dir / "Processed" / "LiveBees" / "3-LiveWingCrops" 

DEBUG = False

In [7]:
def get_lower_y(marker_box):
    # Find the y-coordinates of all four corners of the rectangle
    y_coords = marker_box[:, 1]
    return np.mean(y_coords) 
    

def find_markers(inv_thresh, all_contours, image):
    large_marker_contours = [cnt for cnt in all_contours if (100000 > cv2.contourArea(cnt) > 20000)]
        
    # Create a copy of the image to draw on
    marker_contours = image.copy()

    # length of the line 
    marker_length = []
    
    # Loop over contours 
    for marker_contour in large_marker_contours:
        # Get the minimum-area rectangle for each contour
        marker_rect = cv2.minAreaRect(marker_contour)

        # Extract the width and height of the rectangle
        (center_x, center_y), (length, width), angle = marker_rect
        
        # Ensure length is the longer side
        if length < width:
            length, width = width, length
            # Adjust angle when swapping length and width
            angle = angle + 90  

        # Normalize the angle to 90 degree range (all angles are now between 0 and 90 degrees)
        angle = angle % 90

        # Extract the box points and convert them to integers
        marker_box = cv2.boxPoints(marker_rect)
        marker_box = np.intp(marker_box)
        
        # Filter rectangles
        if (800 > length > 500) and (150 > width) and ((5 >= angle >= 0) or (90 >= angle >= 85)):
            marker_length.append(length)
            # Draw the rectangle on the output image
            cv2.drawContours(marker_contours, [marker_box], 0, (0, 0, 255), 20)
        else:
            cv2.drawContours(marker_contours, [marker_box], 0, (255, 0, 0), 20)

    return marker_length, marker_contours


def find_wing(inv_thresh, all_contours, image):
    large_wing_contours = [cnt for cnt in all_contours if (1000000 > cv2.contourArea(cnt) > 100000)]

    # Create a copy of the image to draw on
    wing_contours = image.copy()

    # Save wing contours
    wing_boxes = []
    wing_rects = []
    # Store pairs of (wing_box, wing_rect)
    wing_data = [] 
    
    # Loop over contours and find the minimum-area bounding rectangle
    for wing_contour in large_wing_contours:
        # Get the minimum-area rectangle for each contour
        wing_rect = cv2.minAreaRect(wing_contour)
        wing_rects.append(wing_rect)
        
        # Extract the box points and convert them to integers
        wing_box = cv2.boxPoints(wing_rect)
        wing_box = np.intp(wing_box)
        wing_boxes.append(wing_box)
        
        # Store the pair
        wing_data.append((wing_box, wing_rect))
    
    # If no valid contours are found, return early
    if len(large_wing_contours) == 0:
        return None, None, wing_boxes, wing_contours
    
    # Identify the lower rectangle (box with the lowest center y-coordinate)
    def get_lower_y(wing_box):
        return np.mean(wing_box[:, 1])  # Average y-coordinate of box points
    
    # Find the pair for the lower rectangle
    lower_rectangle_box, lower_rectangle_rect = max(wing_data, key=lambda data: get_lower_y(data[0]))
    
    # Visualization
    for wing_box in wing_boxes:
        cv2.drawContours(wing_contours, [wing_box], -1, (255, 0, 0), 20)  
    
    # Highlight the lower rectangle in red
    cv2.drawContours(wing_contours, [lower_rectangle_box], -1, (0, 0, 255), 20)
    
    # Return the lowest rectangle box, its matching rect, and the contour image
    return lower_rectangle_box, lower_rectangle_rect, wing_boxes, wing_contours


def crop_wing(wing_box, wing_rect, image):
    # Extract the width and height of the rectangle
    center_rect, (height_rect, width_rect), angle_rect = wing_rect

    # Swap width and height if necessary to make the longer side horizontal
    if height_rect < width_rect:
        angle_rect += 90
    
    # Get the rotation matrix
    rotation_matrix = cv2.getRotationMatrix2D(center_rect, angle_rect, 1.0)
    
    # Rotate the entire image to align the rectangle horizontally
    image_height, image_width = image.shape[:2]
    rotated_wing_image = cv2.warpAffine(image, rotation_matrix, (image_width, image_height), flags=cv2.INTER_LINEAR, borderValue=(255, 255, 255))
    
    # Calculate the bounding box of the rotated rectangle in the rotated image
    x, y, w, h = cv2.boundingRect(np.intp(cv2.transform(np.array([wing_box]), rotation_matrix))[0])
    
    # Crop the aligned rectangle with white padding for any areas outside the original image
    t = 150
    cropped_wing_image = rotated_wing_image[y-t:y+h+t, x-t:x+w+t]
    
    return cropped_wing_image


def preprocessing_main(gray, image, jpg_basename,  output_file):
    global wing_warnings
    global marker_warnings

    # Apply Gaussian Blur
    blurred_image = cv2.GaussianBlur(gray, (5, 5), 0)

    all_marker_lengths = []
    all_wing_crops = []
    threshold = 250
    while threshold >= 0:
        # Apply thresholding to get a binary image
        _, thresh = cv2.threshold(blurred_image, threshold, 255, cv2.THRESH_BINARY)
    
        # Invert the binary image
        inv_thresh = cv2.bitwise_not(thresh)
        
        # Fill holes in the mask
        inv_thresh = binary_fill_holes(inv_thresh).astype(np.uint8) 

        # Scale to match the binary image format
        inv_thresh = inv_thresh * 255

        # Find contour
        all_contours, _ = cv2.findContours(inv_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw contours on the image for visualization
        label_contour_image = cv2.cvtColor(inv_thresh, cv2.COLOR_GRAY2RGB)
        cv2.drawContours(label_contour_image, all_contours, -1, (255, 0, 0), 20)
        
        # Find markers
        marker_length, marker_contours = find_markers(inv_thresh, all_contours, image)   
        all_marker_lengths += marker_length
        
        # Find wing
        wing_box, wing_rect, wing_rects, wing_contours = find_wing(inv_thresh, all_contours, image)

        # Crop wing
        if wing_rect is not None:
            cropped_wing_image = crop_wing(wing_box, wing_rect, image)
            if cropped_wing_image.any():
                all_wing_crops.append(cropped_wing_image)
            
        # Decrease threshold until 0 is reached 
        threshold -= 10

    if len(all_wing_crops) >= 1:
        wing = all_wing_crops[len(all_wing_crops) // 2]        
        wing = Image.fromarray(wing)
        wing = wing.rotate(180)
        wing.save(output_file)
    else:
        wing_warnings.append(jpg_basename)
        
    if len(all_marker_lengths) >= 1:
        # Return the median marker length
        median_length = statistics.median(all_marker_lengths)
        return median_length
    else:
        marker_warnings.append(jpg_basename)
        return None

In [8]:
# Find all jpg files
jpg_files = list(input_dir.glob("*.JPG"))

try:
    # Ensure the input directory exists
    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"Input directory '{input_dir}' was not found.")
    
    # Define output directories
    output_subdir = output_dir / "Wings"
    if os.path.exists(output_subdir):
        print("WARNING: Output directory already exists.")
    os.makedirs(output_subdir, exist_ok=True)
    failed_marker_subdir = output_dir / "Failed" / "NoMarker"
    failed_wing_subdir = output_dir / "Failed" / "NoWing"

    # Empty list for marker length table
    markers = []
    
    # Global warnings counter
    marker_warnings = []
    wing_warnings = []
    
    # Process every file
    for jpg_file_path in tqdm(jpg_files, desc="Processing files", ncols=145):
        jpg_basename = os.path.basename(jpg_file_path)
        output_file = output_subdir / jpg_basename
        failed_marker_file = failed_marker_subdir / jpg_basename
        failed_wing_file = failed_wing_subdir / jpg_basename
        relative_jpg_path = jpg_file_path.relative_to(input_dir)
        
        # Load image
        image = cv2.imread(jpg_file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        # Crop pixels from each side
        padding = 50
        height, width, _ = image.shape
        image = image[padding:height-padding, padding:width-padding]
    
        # Grayscale image
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
        # Run the main function
        marker_warnings_before = len(marker_warnings)
        wing_warnings_before = len(wing_warnings)
        marker_length = preprocessing_main(gray, image, jpg_basename, output_file)
    
        # Save original file if marker or wing could not be identified
        image = Image.fromarray(image)
        if len(marker_warnings) > marker_warnings_before:
            os.makedirs(failed_marker_subdir, exist_ok=True)
            image.save(failed_marker_file)
        if len(wing_warnings) > wing_warnings_before:
            os.makedirs(failed_wing_subdir, exist_ok=True)
            image.save(failed_wing_file)
    
        # Append to results
        markers.append({"Filename": jpg_basename, "MarkerLengthInPixels": marker_length})
    
    # Save to Excel
    df = pd.DataFrame(markers)
    output_excel_path = output_dir / "MarkerLenghts.csv"
    df.to_csv(output_excel_path, index=False)
    
    # Print Total Warnings
    if wing_warnings:
        print(f"WARNING: No Wings Identified:\n\t{"\n\t".join(wing_warnings)}")
    if marker_warnings:
        print(f"WARNING: No Markers Identified:\n\t{"\n\t".join(marker_warnings)}")

# Handle exceptions
except FileNotFoundError as e:
    print(e)
    
except KeyboardInterrupt:
    pass

Processing files: 100%|████████████████████████████████████████████████████████████████████████████████████| 1194/1194 [1:58:15<00:00,  5.94s/it]

	Round01-Hive01-2024_06_19-h01b21.JPG
	Round01-Hive02-2024_06_18-h02b12.JPG
	Round01-Hive02-2024_06_18-h02b44.JPG
	Round01-Hive04-2024_06_17-h04b07.JPG
	Round01-Hive04-2024_06_17-h04b13.JPG
	Round01-Hive05-2024_06_13-h05b02.JPG
	Round01-Hive05-2024_06_13-h05b33.JPG
	Round02-hive11-2024_06_18-h11b04.JPG
	Round02-hive11-2024_06_18-h11b05.JPG
	Round02-hive11-2024_06_18-h11b13.JPG
	Round02-hive11-2024_06_18-h11b17.JPG
	Round02-hive11-2024_06_18-h11b19.JPG
	Round02-hive11-2024_06_18-h11b21.JPG
